# Lesson 2.4: Sequential Chains

---

In real-world LLM applications, it's rare for a task to be completed with a single LLM call. More often, you need to perform a sequence of processing steps, where the output of one step becomes the input for the next. **Sequential Chains** in LangChain are designed to manage these multi-step workflows efficiently and in an organized manner.

## 1. Concept of Sequential Chains: Connecting Multiple Sequential Processing Steps

### 1.1. What are Sequential Chains?

**Sequential Chains** are a type of Chain in LangChain that allows you to connect multiple processing steps (each step can be a Chain or another Runnable) in a sequential order. The output of one step is automatically passed as input to the next step in the chain.

* **Relationship:** In Lesson 2.2, we saw how LCEL allows chaining single `Runnables` (Prompt, LLM, Parser) using the `|` operator. Sequential Chains extend this idea, allowing you to chain *Chains* or *more complex processing flows* together.

### 1.2. Why are Sequential Chains Needed?

* **Complexity Management:** Simplify the management of complex workflows by breaking them down into smaller, more manageable steps.
* **Reusability:** Each step in the chain can be an independent Chain and can be reused elsewhere.
* **Clear Data Flow:** Helps visualize and understand how data flows through your application.
* **Error Reduction:** Reduces the likelihood of errors due to manual input/output connections between steps.




---

## 2. `SimpleSequentialChain`: Connecting LLMChains in a Simple Sequence

`SimpleSequentialChain` is the simplest way to connect multiple `LLMChain`s (or other chains) together.

### 2.1. `SimpleSequentialChain` Concept

* **Characteristics:**
    * Has only **one single input** and **one single output** for the entire chain.
    * The output of each step is automatically passed as input to the next step.
    * No ability to name intermediate inputs/outputs.
* **When to Use:** Suitable for very simple, linear workflows where you just need to pass a text string from one step to the next without managing multiple variables.

### 2.2. `SimpleSequentialChain` Example (Illustrative, LCEL is Preferred)

While `SimpleSequentialChain` still exists, **LCEL (LangChain Expression Language)** with the `|` operator is the more modern and flexible way to build sequential chains. However, to illustrate the concept, we will look at this example.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Step 1: Create a Chain to summarize text
prompt_summary = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a summarization assistant. Summarize the following paragraph into a single sentence."),
    HumanMessage(content="{text}"),
])
chain_summary = prompt_summary | llm | StrOutputParser()

# Step 2: Create a Chain to generate questions from the summary
prompt_question = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a teacher. From the following summary, create one multiple-choice question with 4 options A, B, C, D and the correct answer."),
    HumanMessage(content="Summary: {text}"), # Note: input here is 'text'
])
chain_question = prompt_question | llm | StrOutputParser()

# Connect the two Chains using SimpleSequentialChain
# The output of chain_summary will automatically become the input of chain_question
# The input variable name of chain_question must match the output variable name of chain_summary (default is 'text')
overall_chain_simple = SimpleSequentialChain(
    chains=[chain_summary, chain_question],
    verbose=True # To see execution steps
)

# Execute the chain
input_text = """
Artificial intelligence (AI) is changing the world at a rapid pace. From self-driving cars to virtual assistants, AI is permeating every aspect of life.
One of the biggest breakthroughs in AI recently is the development of Large Language Models (LLMs). These models, such as OpenAI's GPT-4 or Google's Gemini, are capable of understanding and generating natural language in an astonishing way.
However, building practical applications with LLMs is not simple. Developers often face challenges in prompt management, connecting LLMs to external data, and building complex logical chains. This is where LangChain comes into play.
LangChain is an open-source framework that simplifies this process, providing tools to connect LLMs with data sources, tools, and other components, allowing for the construction of more powerful and flexible AI applications.
"""

print("--- SimpleSequentialChain Output ---")
response_simple = overall_chain_simple.invoke(input_text) # Input is a text string
print(response_simple)


---

## 3. `SequentialChain`: Connecting More Complex Chains with Named Inputs/Outputs

`SequentialChain` provides more granular control over the data flow between steps.

### 3.1. `SequentialChain` Concept

* **Characteristics:**
    * Allows you to define multiple inputs and outputs for the entire chain.
    * You can explicitly specify which outputs from one step will be passed as inputs to the next step by naming the variables.
    * Supports passing multiple input variables to a single step.
* **When to Use:** Suitable for more complex workflows where you need to manage multiple variables, or when you want some intermediate outputs to be retained as the final output of the entire chain.

### 3.2. `SequentialChain` Example (Illustrative, LCEL is Preferred)

Similar to `SimpleSequentialChain`, `SequentialChain` is also a traditional way of building chains. With the advent of LCEL, building complex chains with named inputs/outputs has become much easier. We will not delve into the old `SequentialChain` syntax but will focus on how to achieve this using LCEL.


---

## 4. Practical Examples of Building Sequential Chains (using LCEL)

With **LangChain Expression Language (LCEL)**, building sequential chains becomes much more intuitive and powerful. You can easily connect `Runnables` and control data flow using objects like `RunnablePassthrough` and `RunnableParallel`.

### 4.1. Example 1: Summarize an Article, then Generate Multiple-Choice Questions from the Summary

This is a classic example of a sequential chain, where the output of the summarization step is used as input for the question generation step.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# --- Step 1: Summarization Chain ---
summarize_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a summarization assistant. Summarize the following paragraph into 3-4 key sentences."),
    HumanMessage(content="Paragraph: {article_text}"),
])
summarize_chain = summarize_prompt | llm | StrOutputParser()

# --- Step 2: Question Generation Chain from Summary ---
question_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a teacher. From the following summary, create 3 multiple-choice questions with 4 options A, B, C, D and indicate the correct answer for each. Each question and answer must be clear."),
    HumanMessage(content="Summary: {summary_text}"),
])
question_chain = question_prompt | llm | StrOutputParser()

# --- Connect Chains using LCEL ---
# Use RunnablePassthrough to pass the initial input (article_text)
# and then use RunnableParallel to route the output of summarize_chain
# as input for question_chain
full_qa_chain = (
    {"summary_text": summarize_chain, "original_text": RunnablePassthrough()}
    | question_chain
)

# Note: "original_text" here is just to illustrate RunnablePassthrough,
# it is not used in question_chain. If you want to pass it through,
# you need to include it in the prompt of question_chain.
# A better alternative is:
# full_qa_chain = (
#     {"summary_text": summarize_chain}
#     | question_chain
# )
# Or if you want to retain both summary and questions:
# full_qa_chain = (
#     {"summary_text": summarize_chain, "original_text": RunnablePassthrough()}
#     | RunnableParallel(
#         questions=question_chain,
#         summary=lambda x: x["summary_text"]
#     )
# )


article_to_process = """
Climate change is one of the biggest challenges facing humanity in the 21st century. It refers to long-term shifts in temperatures and weather patterns. These shifts may be natural, but since the 1800s, human activities have been the main driver of climate change, primarily due to the burning of fossil fuels like coal, oil, and gas, which produces heat-trapping gases.

The impacts of climate change include rising sea levels, more frequent and intense extreme weather events (storms, droughts, floods), the melting of glaciers and polar ice, and shifts in natural ecosystems. These threaten biodiversity, food security, and human health.

To combat climate change, urgent global actions are needed. Solutions include reducing greenhouse gas emissions by transitioning to renewable energy, improving energy efficiency, protecting forests, and developing carbon capture technologies. Additionally, adapting to unavoidable changes is also a crucial part of the strategy.
"""

print("--- Summarize Article and Generate Multiple-Choice Questions ---")
response_qa = full_qa_chain.invoke({"article_text": article_to_process})
print(response_qa)

**Data Flow Explanation in `full_qa_chain`:**

1.  `{"summary_text": summarize_chain, "original_text": RunnablePassthrough()}`:
    * This is an implicit `RunnableParallel`, allowing for concurrent processing or input routing.
    * `summarize_chain`: Receives the input `{"article_text": article_to_process}` from `invoke` and executes the summarization chain, returning the summary. This result is assigned to the key `summary_text`.
    * `RunnablePassthrough()`: Receives the entire original input (`{"article_text": article_to_process}`) and passes it through. This result is assigned to the key `original_text`.
    * The output of this step is a dictionary: `{"summary_text": "...", "original_text": "..."}`.
2.  `| question_chain`:
    * `question_chain` expects an input with the key `summary_text`.
    * It will automatically take the value of `summary_text` from its input dictionary and use it to generate questions.
    * `original_text` (if present) will be ignored because `question_chain` does not require it.

### 4.2. Example 2: Translate a Passage, then Check the Grammar of the Translated Passage

This example illustrates passing results from one step (translation) to the next (grammar check).

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# --- Step 1: Translation Chain ---
translate_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a professional translator. Translate the following text from {source_lang} to {target_lang}."),
    HumanMessage(content="Text: {text_to_translate}"),
])
translate_chain = translate_prompt | llm | StrOutputParser()

# --- Step 2: Grammar Check Chain ---
grammar_check_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a grammar expert. Check the grammar of the following text. If there are errors, correct them and briefly explain the error. If there are no errors, say 'No grammatical errors.'"),
    HumanMessage(content="Text: {translated_text}"),
])
grammar_check_chain = grammar_check_prompt | llm | StrOutputParser()

# --- Connect Chains using LCEL ---
# Use RunnableParallel to pass both the original text and the translated result to the grammar check chain
# Or simpler, if only the translated result is needed:
# full_translation_grammar_chain = translate_chain | grammar_check_chain

# To retain both original and translated text, and the grammar check result:
full_translation_grammar_chain = (
    {
        "translated_text": translate_chain,
        "original_text_input": RunnablePassthrough() # Retain the original input
    }
    | grammar_check_chain
)

# Note: "original_text_input" here refers to the entire input dictionary of the parent chain.
# If you want to pass only a part of the original input, you need to select it.
# E.g.: {"translated_text": translate_chain, "original_text": lambda x: x["text_to_translate"]}

text_to_process = "I am go to the market yesterday."
source_lang_input = "English"
target_lang_input = "Vietnamese"

print("--- Translate and Grammar Check ---")
response_grammar = full_translation_grammar_chain.invoke({
    "text_to_translate": text_to_process,
    "source_lang": source_lang_input,
    "target_lang": target_lang_input
})
print(response_grammar)

**Data Flow Explanation in `full_translation_grammar_chain`:**

1.  `{ "translated_text": translate_chain, "original_text_input": RunnablePassthrough() }`:
    * `translate_chain`: Receives the input `{"text_to_translate": ..., "source_lang": ..., "target_lang": ...}` and executes the translation chain, returning the translated text. This result is assigned to the key `translated_text`.
    * `RunnablePassthrough()`: Receives the entire original input dictionary and passes it through, assigning it to the key `original_text_input`.
    * The output of this step is a dictionary: `{"translated_text": "...", "original_text_input": {...}}`.
2.  `| grammar_check_chain`:
    * `grammar_check_chain` expects an input with the key `translated_text`.
    * It will automatically take the value of `translated_text` from its input dictionary and use it to check grammar.
    * `original_text_input` will be ignored because `grammar_check_chain` does not require it.


---

## Lesson Summary

This lesson introduced **Sequential Chains** in LangChain, a crucial concept for building multi-step LLM applications. We learned about `SimpleSequentialChain` as a straightforward way to connect sequential steps, despite its limitations in input/output management. The main focus of the lesson was on using **LangChain Expression Language (LCEL)** with the `|` operator to build sequential chains more flexibly and powerfully, allowing granular control over data flow between components. Through practical examples of summarizing and question generation, as well as translation and grammar checking, you've seen how LCEL helps organize and execute complex workflows where the output of one step becomes the input for the next.